# step0: import the simulator

In [1]:
#import pyNN.spiNNaker as sim # set accordly also the kernel in the notebook
import spynnaker8 as sim

from pyNN import space 

import numpy as np
import matplotlib.pyplot as plt

# note that if you use nest, import pyNN.nest as sim (see http://neuralensemble.org/docs/PyNN/backends.html)

Detected PyNN version 0.9.4 and Neo version 0.6.1


# step1: setup the simulator

In [2]:
sim.setup(
    timestep=1, # [ms]
    min_delay=1, # [ms]
    max_delay=1*144) # [ms] # not that the max_delay supported by SpiNNaker is timestep * 144


2022-01-05 16:48:24 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2022-01-05 16:48:24 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2022-01-05 16:48:24 WARNING: /home/spinnaker/CNT-2022/SpiNNaker/reports has 22 old reports that have not been closed
2022-01-05 16:48:24 WARNING: /home/spinnaker/CNT-2022/SpiNNaker/application_generated_data_files has 22 old reports that have not been closed
2022-01-05 16:48:24 INFO: Setting time scale factor to 1.
2022-01-05 16:48:24 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


0

# step2: the cell types 

In [4]:
# define your own pars, 
# dictonary of testing parameters Brette et al. (2007) Journal of Computational Neuroscience 23: 349-398

testing_pars = {'tau_m': 20, 
                'cm': 1.0, 
                'v_rest': -60.0, 
                'v_reset': -60.0, 
                'v_thresh': -50.0, 
                'tau_syn_E': 5.0, 
                'tau_syn_I': 10.0, 
                'tau_refrac': 5, 
                'i_offset': 0.0, 
                'e_rev_E': 0.0, 
                'e_rev_I': -80.0}

# step3: making cell populations


In [5]:
# define dictionaries and settings

pops = {} # populations of exc and inh cells
n_cells = {} # specific number of exc or inh cells in pops
neuron_Model = {} # model of neuron adopted for the simulation
neuron_ModelPars = {} # model parameters 
initial_Values = {} # initial values of state variables (membrane potential and conductances)

In [6]:
# define cells quantities

n = 1500 # total number of cells in the network
r_ei = 4.0 # ratio exc and inh cells, i.e. every 4 exc cells there's 1 inh cell

In [7]:
# define the popolation of excitatory neurons 

popName = 'exc'

n_cells[popName] = int(round((n * r_ei / (1 + r_ei))))
initial_Values[popName] = sim.IF_cond_exp.default_initial_values

pops[popName] = sim.Population(n_cells[popName], 
                               sim.IF_cond_exp(**testing_pars),
                               label = popName)

pops[popName].record(['spikes','v','gsyn_exc','gsyn_inh'])   

print('the total number of excitatory cells is: ', pops['exc'].size)

the total number of excitatory cells is:  1200


In [8]:
# define a popolation of inhibitory neurons in 1D

popName = 'inh'

n_cells[popName] = n - n_cells['exc']  # number of inhibitory cells
initial_Values[popName] = sim.IF_cond_exp.default_initial_values

pops[popName] = sim.Population(n_cells[popName], 
                               sim.IF_cond_exp(**testing_pars),
                               label = popName)

pops[popName].record(['spikes','v','gsyn_exc','gsyn_inh']) 

print('the total number of inhibitory cells is: ', pops['inh'].size)

the total number of inhibitory cells is:  300


In [9]:
# making the connections for a balanced network on inhibitor and excitatory cells in 1D

from pyNN.random import NumpyRNG, RandomDistribution
sim.set_number_of_neurons_per_core(sim.IF_curr_exp, 100)      # this will set
#  100 neurons per core
sim.set_number_of_neurons_per_core(sim.IF_cond_exp, 50)      # this will set

pconn = 0.02
rngseed = 98766987
rng = NumpyRNG(seed=rngseed, parallel_safe=True)

exc_cells = pops['exc']
inh_cells = pops['inh']
Gexc = 4.     # (nS)
Ginh = 51.    # (nS)
w_exc = Gexc * 1e-3              # We convert conductances to uS
w_inh = Ginh * 1e-3
delay = 2 # ms

exc_conn = sim.FixedProbabilityConnector(pconn, rng=rng)
inh_conn = sim.FixedProbabilityConnector(pconn, rng=rng)

connections = {
    
    'e2e': sim.Projection(
        exc_cells, 
        exc_cells, 
        exc_conn, 
        receptor_type='excitatory',
        synapse_type=sim.StaticSynapse(weight=w_exc, delay=delay)),
    
    'e2i': sim.Projection(
        exc_cells, 
        inh_cells,
        exc_conn,
        receptor_type='excitatory',
        synapse_type=sim.StaticSynapse(weight=w_exc, delay=delay)),
    
    'i2e': sim.Projection(
        inh_cells,
        exc_cells, 
        inh_conn, 
        receptor_type='inhibitory',
        synapse_type=sim.StaticSynapse(weight=w_inh, delay=delay)),
    
    'i2i': sim.Projection(
        inh_cells, 
        inh_cells, 
        inh_conn,
        receptor_type='inhibitory',
        synapse_type=sim.StaticSynapse(weight=w_inh, delay=delay))

        }

connections
rngseed = 98766987

rng = NumpyRNG(seed=rngseed, parallel_safe=True)
uniformDistr = RandomDistribution('uniform', [-60.0, -50], rng=rng)
exc_cells.initialize(v=uniformDistr)
inh_cells.initialize(v=uniformDistr)

# step7: setting the thalamic stimulus

In [10]:
# otherwise define a spike sources from poisson process

do_run = True

if do_run:
    stim_dur = 50.    # (ms) duration of random stimulation
    rate = 100.       # (Hz) frequency of the random stimulation
    
    ext_stim = sim.Population(
        20, sim.SpikeSourcePoisson(rate=rate, duration=stim_dur),
        label="expoisson")
    
    rconn = 0.01
    ext_conn = sim.FixedProbabilityConnector(rconn)
    ext_stim.record("spikes")

    connections['ext2e'] = sim.Projection(
        ext_stim, 
        exc_cells, 
        ext_conn,
        receptor_type='excitatory',
        synapse_type=sim.StaticSynapse(weight=0.1))
    
    connections['ext2i'] = sim.Projection(
        ext_stim, 
        inh_cells,
        ext_conn,
        receptor_type='excitatory',
        synapse_type=sim.StaticSynapse(weight=0.1))
    
connections

{'e2e': projection from pre exc to post exc with connector FixedProbabilityConnector(0.02),
 'e2i': projection from pre exc to post inh with connector FixedProbabilityConnector(0.02),
 'i2e': projection from pre inh to post exc with connector FixedProbabilityConnector(0.02),
 'i2i': projection from pre inh to post inh with connector FixedProbabilityConnector(0.02),
 'ext2e': projection from pre expoisson to post exc with connector FixedProbabilityConnector(0.01),
 'ext2i': projection from pre expoisson to post inh with connector FixedProbabilityConnector(0.01)}

# step8: run simulation

In [11]:
simtime = 1000
sim.run(simtime) # time_bins=200 [ms]

# wait some minuts and it prints several info about HW and SW...

2022-01-05 17:04:26 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us
2022-01-05 17:04:26 INFO: Starting execution process
2022-01-05 17:04:29 INFO: Time 0:00:03.118079 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2022-01-05 17:04:38 INFO: Time 0:00:08.716753 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2022-01-05 17:04:43 INFO: Time 0:00:04.855704 taken by PartitionAndPlacePartitioner
Created spalloc job 6233373
2022-01-05 17:04:43 INFO: Created spalloc job 6233373
Waiting for board power commands to complete.
2022-01-05 17:04:43 INFO: Waiting for board power commands to complete.
2022-01-05 17:04:48 INFO: Time 0:00:05.048973 taken by SpallocAllocator
2022-01-05 

1000.0

# step9: save results in a folder

In [12]:
outputs = {}

for layer in ['exc', 'inh']:
    
    # save on the notebook space
    outputs[layer] = pops[layer].get_data()
    
    # save in the folder space
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops[layer].write_data(str(layer) + '_' + str(recording) + '.pkl')
        

Getting spikes for exc
|0%                          50%                         100%|
Getting v for exc
|0%                          50%                         100%|
Getting gsyn_exc for exc
|0%                          50%                         100%|
Getting gsyn_inh for exc
|0%                          50%                         100%|
Getting spikes for exc
|0%                          50%                         100%|
Getting v for exc
|0%                          50%                         100%|
Getting gsyn_exc for exc
|0%                          50%                         100%|
Getting gsyn_inh for exc
|0%                          50%                         100%|
Getting spikes for exc
|0%                          50%                         100%|
Getting v for exc
|0%                          50%                         100%|
Getting gsyn_exc for exc
|0%                          50%                         100%|
Getting gsyn_inh for exc
|0%                          50%  

# step10: recover results

In [13]:
def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(key, analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

# check results
results = recover_results(outputs)
results.keys()

exc v
exc gsyn_exc
exc gsyn_inh
inh v
inh gsyn_exc
inh gsyn_inh


dict_keys([('exc', 'v'), ('exc', 'gsyn_exc'), ('exc', 'gsyn_inh'), ('exc', 'spikes'), ('inh', 'v'), ('inh', 'gsyn_exc'), ('inh', 'gsyn_inh'), ('inh', 'spikes')])

# step11: postprocessing (looking the results)

In [14]:
exc_spikes = exc_cells.get_data("spikes")
from pyNN.utility.plotting import Figure, Panel
%matplotlib
Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(exc_spikes.segments[0].spiketrains, xlabel="Time/ms", xticks=True,
          yticks=True, markersize=0.2, xlim=(0, simtime)),
    title="Vogels-Abbott benchmark: excitatory cells spikes",
    annotations="Simulated with {}".format(sim.name())
)


Getting spikes for exc
|0%                          50%                         100%|

Using matplotlib backend: module://ipympl.backend_nbagg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
inh_spikes = inh_cells.get_data("spikes")
from pyNN.utility.plotting import Figure, Panel
%matplotlib
Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(inh_spikes.segments[0].spiketrains, xlabel="Time/ms", xticks=True,
          yticks=True, markersize=0.2, xlim=(0, simtime)),
    title="Vogels-Abbott benchmark: inhibitory cells spikes",
    annotations="Simulated with {}".format(sim.name())
)

Getting spikes for inh
|0%                          50%                         100%|

Using matplotlib backend: module://ipympl.backend_nbagg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# check spikes

#exc_cells.get_spike_counts()
# raster plot and spike counts

In [ ]:
# check the voltage

#ts and hist

In [ ]:
# check the conductances

# ts and hist

# step12: close simulation


In [ ]:
sim.end()

# learn-by-examples 

- [1D entry network](eg_1D_entry-network.ipynb)
- [1D testing cell models network](eg_1D_testing-cell-models-network.ipynb)
- [1D testing STDP model network](eg_1D_testing-STDP-model-network.ipynb)
- [1D small-world network](eg_1D_small-world-network.ipynb)
- [1D decaying network](eg_1D_decaying-network.ipynb)
- [1D persistent network](eg_1D_persistent-network.ipynb)
- [1D diverging network](eg_1D_diverging-network.ipynb)





# learn-with-exercices and resume with link for the afternoon 
- [ex_network_A](ex_network_A.ipynb), i.e., a 1D network of 100 cells that receives a sequence of random thalamic inputs
- [ex_network_B](ex_network_B.ipynb), i.e., a 2D network of 900 excitatory cells that receives a sequence of 2D geometric thalamic inputs
- read the material and playing with examples
- resources:
    - SpiNNaker Jupyter Lab login at https://spinn-20.cs.man.ac.uk/hub/home
    - clone repository https://github.com/albertoarturovergani/CNT-2021
    - SpiNNaker


# learn-from-scratch

- create equivalent or extended PyNN functions by using numpy, e.g., specially for peculiar type of connectivities (use [FromListConnector](http://neuralensemble.org/docs/PyNN/reference/connectors.html#pyNN.connectors.FromListConnector))
- create a network and check the [balance](http://www.scholarpedia.org/article/Balance_of_excitation_and_inhibition) between excitatory and inhibitory components (e.g. [Brette et al 2007](http://neuralensemble.org/docs/PyNN/examples/VAbenchmarks.html))
- explore boundary conditions in 1D by comparing open and close ring networks (define properly the list of connections)
- compare two populations of different cell models and observe their difference on reacting to a stimulus
- create a network to make a deep signal processing with statistics, data analysis, and exploration of the frequency domain (FFT, HHT)
- design a set of stimuli with different onset timing and shape and test them on a simple network
- take one [example](http://neuralensemble.org/docs/PyNN/examples.html) from the PyNN website made for NEST and adapt to run on SpiNNaker
- take one example from the learn-by-examples section and extend it in 2D or 3D  
- use the notebook called [eg_nD_blank_network](eg_nD_blank-network.ipynb) and start your personal simulations!
